# Where are the sushi bars?

### Motivation

The focus of my PhD research was quantitative coral reef research, and I stay in close touch with former collaborators. Erik Franklin, a researcher at UH-Manoa, is thinking about the connection between ocean ecosystems and society, and he's interested in sushi sustainability. There are several researchers and organizations working in this area, but it's surprisingly hard to get answers to simple questions. For instance, how many sushi restaurants are there in the US and where are those restaurants located?

Here, I'm going to walk through how we gathered some basic data on sushi restaurants in the US.

### Failed approaches

One of the obvious places to get this information is from Yelp, and the easiest way is by scraping data from their website directly. If someone wanted to try this approach, they would probably know it's against Yelp's ToS and try to get creative by writing a script to make requests in a rate-limited and randomized fashion, to try and approximate human searches. They might do this because it's dead simple to search all zip codes and page through all of the results, but they'd also find out that Yelp is pretty good at identifying automated scrapers. I, of course, would never do such a thing.

Another approach would be to use the Yelp API to search for sushi restaurants by zip code. Unfortunately, the API methods are not engineered in a way that facilitates exhaustive searches. The API methods are best for answering questions like, "What are a few restaurants close to my location?" or "What additional information do you have for a given restaurant?", rather than questions like "How many restaurants are within this area?". This is primarily because the Yelp API only returns up to 40 businesses within 40,000 meters of a given location, meaning that multiple queries are needed when the requester is interested in areas with more than 40 relevant businesses or distances greater than 40,000 meters.

### Implementing a recursive, grid-based, search algorithm

My solution? The Yelp API has a method for bounded searches, where one can request a list of businesses within a lat/lon box. I can divide the US into a grid where the distance from the center of each grid cell to any point within the cell is smaller than the 40,000 meter limit. I can then query the Yelp API for a list of sushi restaurants within each of those grid cells. Whenever I come to a cell where there are 40 restaurants in the response, I assume that there are more restaurants in that location than can be returned by the API; I then subdivide the cell into smaller cells and repeat the queries until I have fewer than 40 restaurants in each response and the search results are comprehensive.

How will that work in practice? Grid cells in Wyoming, a landlocked state with the lowest population density in the continental US, are likely to have very few sushi restaurants and not need any recursion. The grid cells covering San Francisco are likely to have many more than 40 sushi restaurants, and it's likely that those areas will need to be subdivided multiple times to get full coverage.

### Let's see the code

I generally write Python scripts with the following ordering:
- imports
- global or hidden constants and parameters
- high-level method to do everything
- increasingly lower-level methods ordered by usage and/or level of complexity

While this format makes it easy to maintain code, it's not ideal for demonstrating code development. Instead, I'll try to narrate code development in a somewhat logical order.

First, I need a local file to save my data periodically, as well as convenience methods for saving and loading that data.

In [1]:
import json


# Local file for saving data results
PATH_DATA_MASTER = '/Users/nsfabina/yelp_sushi/data_master.json'


def save_master_data(data_master):
    with open(PATH_DATA_MASTER, 'w') as file_:
        json.dump(data_master, file_)
        
        
def load_master_data():
    with open(PATH_DATA_MASTER) as file_:
        data_master = json.load(file_)
    return data_master


# Create empty file
save_master_data(dict())

Second, I'm going to use a third-party library to authenticate with the Yelp servers, and I need to set some search parameters.

In [2]:
import yelp
from yelp.client import Client
from yelp.oauth1_authenticator import Oauth1Authenticator


# Yelp auth - obscured
CONSUMER_KEY = '*'
CONSUMER_SECRET = '*'
TOKEN = '*'
TOKEN_SECRET = '*'
YELP_CLIENT = Client(Oauth1Authenticator(
    consumer_key=CONSUMER_KEY, consumer_secret=CONSUMER_SECRET, token=TOKEN,
    token_secret=TOKEN_SECRET))

# Yelp API params
API_SORT = '1'
API_CATEGORY_FILTER = 'sushi'
API_PARAMS = {'sort': API_SORT, 'category_filter': API_CATEGORY_FILTER}

Third, I need to set location parameters for my grid-based search.

In [3]:
# Brute force US latlon ranges - because it's not worth the extra investment to check whether
# a grid point is on US territory or in the middle of the ocean
MIN_LAT = 17.75
MAX_LAT = 49.25
MIN_LON = -124.75
MAX_LON = -62.50

# Distance conversions
# Note that I just assume constant degrees from the equator to the poles, even though longitude 
# degrees cover more area closer to the equator. Instead, I use values for degree conversion from
# the equator and realize that my grid will have more cells than necessary from east to west.
# I'm also setting a buffer because I don't know whether Yelp's distance limit is as the crow
# flies, as the car drives, or whether some crazy engineer came up with a wonkier definition, so
# I'm just going to play it safe.
API_DISTANCE_LIMIT_METERS = 40000.0
DEGREES_TO_METERS = 111111.11
GLOBAL_GRID_DISTANCE = API_DISTANCE_LIMIT_METERS / DEGREES_TO_METERS
BUFFER_DISTANCE = 0.5

I'm going to use a GridPoint class to hold all of the location information for the global and location gridpoints, and that class will calculate search bounds knowing just the central lat/lon and resolution level.

In [4]:
class GridPoint(object):
    def __init__(self, lon_center, lat_center, resolution):
        self.lon_center = lon_center
        self.lat_center = lat_center
        self.resolution = resolution
        self._set_bounds()
    
    def _set_bounds(self):
        distance = GLOBAL_GRID_DISTANCE / self.resolution / 2.0  # Half the distance across the grid cell
        self.bounds = {
            'min_lat': self.lat_center-distance, 'min_lon': self.lon_center-distance,
            'max_lat': self.lat_center+distance, 'max_lon': self.lon_center+distance,
        }  

The global grid will be an iterator of GridPoints across the US, with latitude and longitude step sizes defined by the Yelp distance limit. I'm going to use numpy to generate those degree vectors, even though it's totally possible to do that with vanilla Python.

In [5]:
import numpy as np


def create_global_grid():
    resolution_start = 1.0
    return iter(GridPoint(lon, lat, resolution_start)
                for lon in np.arange(MIN_LON, MAX_LON, BUFFER_DISTANCE * GLOBAL_GRID_DISTANCE)
                for lat in np.arange(MIN_LAT, MAX_LAT, BUFFER_DISTANCE * GLOBAL_GRID_DISTANCE))

The local grids will be iterators of GridPoints as well. Local grids will be created whenever a grid cell has more restaurants than can be returned by the Yelp API. In this case, I subdivide a grid cell into four, equally-sized subcells by dividing it in half north-south and east-west. I also need to define the new grid cells as having greater resolution so that the bounds are adjusted as necessary.

In [6]:
def create_local_grid(point):
    distance = GLOBAL_GRID_DISTANCE / point.resolution / 2.0  # Half the distance across the grid cell
    local_distance = distance / 2.0  # Midpoint for splitting the grid cell into four points
    return iter(GridPoint(point.lon_center + (local_distance * x),
                          point.lat_center + (local_distance * y),
                          point.resolution * 2.0)
                for x in (-1, 1) for y in (-1, 1))

The previous components are more-or-less independent from one another, while everything from this point forward is going to be interconnected. For clarity, I'll show the high-level function that I call for each grid cell in the global grid, and then I'll walk through each of the methods it calls. Basically, I build the global grid and iterate through each cell:
- query the Yelp API for the current cell,
- parse the businesses from the response,
- check if the response returned all businesses,
    - if the search was exhaustive, move to the next cell OR
    - subdivide the cell and conduct additional searches

You'll notice this is a recursive algorithm that terminates when the Yelp API has returned all sushi restaurants in an area of interest, and you'll also notice that I update my data object after each action is taken.

In [7]:
def get_businesses_nearby_point(point, data_master):
    response = query_point_for_businesses(point, data_master)
    data_master = update_master_data_points(point, data_master)
    if response is None:
        return data_master
    businesses = parse_businesses(response)
    data_master = update_master_data_businesses(businesses, data_master)
    if check_is_exhausted(response) is False:
        grid_local = create_local_grid(point)
        for recursed_point in grid_local:
            data_master = get_businesses_nearby_point(recursed_point, data_master)
    return data_master

Now, the lower-level methods in the order they're called.

When I query each point, I want to catch any potential errors. I want to stop if I've used all my API calls or if I see an unexpected error, while I just want to log the other, expected Yelp errors. The latter errors are things like, "Yelp screwed up" or "Nick requested data from the middle of the ocean". I'll save data before the program terminates so that I can pick up where I left off, and I'll print points with unexpected errors for easier debugging.

In [8]:
ERRORS_CRITICAL = (yelp.errors.ExceededReqs)
ERRORS_LOG =(yelp.errors.InternalError,
             yelp.errors.InvalidParameter,
             yelp.errors.UnavailableForLocation)


def query_point_for_businesses(point, data_master):
    try:
        response = YELP_CLIENT.search_by_bounding_box(
            point.bounds['min_lat'], point.bounds['min_lon'], point.bounds['max_lat'],
            point.bounds['max_lon'], **API_PARAMS)
    except ERRORS_CRITICAL as error:
        print(error)
        save_master_data(data_master)
        raise error
    except ERRORS_LOG as error:
        data_master = update_master_data_errors(error, point, data_master)
        response = None
    except Exception as error:
        print(point.lon_center, point.lat_center, point.resolution)
        print(error)
        save_master_data(data_master)
        raise error
    return response


When responses are returned successfully, I want to parse all of the business information and convert the Yelp classes to pure JSON for saving. While I could pickle these Yelp classes, there are too many gotchas with pickling that I'd like to avoid, such as changes to the data definitions for Yelp or for my own code. Notice that I'm not going to clean or modify any data, making it easier for me to be flexible with how data is used downstream.

In [9]:
def parse_businesses(response):
    return {business.id: parse_business(business) for business in response.businesses}


def parse_business(business):
    parsed = dict()
    if business.location is not None:
        parsed.update({
            'address': business.location.address,
            'city': business.location.city,
            'country_code': business.location.country_code,
            'postal_code': business.location.postal_code,
            'state_code': business.location.state_code,
            })
    if business.location.coordinate is not None:
        parsed.update({
            'latitude': business.location.coordinate.latitude,
            'longitude': business.location.coordinate.longitude,
            })
    parsed.update({
        'categories': [(cat.alias, cat.name) for cat in business.categories],
        'phone_display': business.display_phone,
        'id': business.id,
        'is_claimed': business.is_claimed,
        'is_closed': business.is_closed,
        'name': business.name,
        'phone': business.phone,
        'url': business.url,
       })
    return parsed

We can be confident that a search is exhausted -- that all restaurants in a grid cell are known -- if there are fewer than 40 businesses in the response. If there are 40 businesses in the response, we can't determine whether the search was exhausted and there were exactly 40 businesses in the grid cell or if the search is incomplete and there were 41 or more businesses. In either case, we need to recurse to find any unknown sushi restaurants.

In [10]:
API_BUSINESS_LIMIT = 40

def check_is_exhausted(response):
    if response.total < API_BUSINESS_LIMIT:
        return True
    return False

Here, I have some convenience methods for updating the data object. Basically, I have a dictionary with keys for businesses, all points in the order they were searched, the last global grid point completed, and the errors that were observed and which points they were associated with. Note that I assume Yelp has unique internal IDs for every business -- a safe assumption -- and that businesses are added to a dictionary with this internal ID as a key. Thus, the final list of businesses with only have unique values.

In [11]:
def update_master_data_businesses(businesses, data_master):
    data_master.setdefault('businesses', {})
    data_master['businesses'].update(businesses)
    return data_master


def update_master_data_points(point, data_master):
    data_master.setdefault('points', [])
    data_master['points'].append(jsonify_point(point))
    return data_master
    

def update_master_data_last_global_point(point_global, data_master):
    data_master['last_complete_global_point'] = jsonify_point(point_global)
    return data_master


def update_master_data_errors(error, point, data_master):
    error_dict = json.loads(str(error).replace('\'', '"').replace('u\"', '"'))
    error_id = error_dict['id']
    data_error = data_master.setdefault('errors', {}).setdefault(error_id, [])
    data_error.append(jsonify_point(point))
    return data_master


We're almost there! I just have a couple of convenience methods for interacting with data. First, two methods for converting points to JSON and back to the class representation...

In [12]:
def jsonify_point(point):
    return {'lat_center': point.lat_center, 'lon_center': point.lon_center,
            'resolution': point.resolution}


def classify_point(point_json):
    return GridPoint(point_json['lon_center'], point_json['lat_center'],
                     point_json['resolution'])

... and finally, a method for finding the starting point in the global grid when (not if) my program inevitably needs to be restarted. Restarts will happen when the API call limit is reached or when I run into unexpected errors.

In [13]:
def iterate_to_starting_global_point(grid_global, data_master):
    last_point = data_master.get('last_complete_global_point')
    if last_point is None:
        return
    last_point = classify_point(last_point)
    for idx, point in enumerate(grid_global):
        if (point.lat_center == last_point.lat_center and 
                point.lon_center == last_point.lon_center and 
                point.resolution == last_point.resolution):
            print('Points skipped: {}'.format(idx))
            break


Not too shabby, eh? The only concern I have is not knowing how long it's going to take to complete my searches. There are at least 65,000 points to search in the national grid, plus as many points in the recursions for exhauastive searches. I have a limit of 25,000 calls per day, so it just depends on how deep I need to go for complete coverage.

Below is my snippet for running this code. The method calls are well-named, but in plain English:
- load the existing data,
- create the global grid,
- find the starting point in the global grid,
- query each point in order
- save the data after each 100 queries

In [ ]:
data_master = load_master_data()
grid_global = create_global_grid()
iterate_to_starting_global_point(grid_global, data_master)
for idx, point_global in enumerate(grid_global):
    print '\r', idx,
    get_businesses_nearby_point(point_global, data_master)
    data_master = update_master_data_last_global_point(point_global, data_master)
    if idx % 100 == 0:
        save_master_data(data_master)

I save after each 100 queries because writes add significant overhead when data objects get larger. The absolute time to save is short, but the time relative to rerunning queries is long.

### What next?

Erik and I are going to create a visualization of sushi restaurants in the US after this is complete, as well as use the raw data for specific research purposes. Keep an eye out for the former in an updated version of this notebook, and the latter in the pages of _Science_.